In [ ]:
# Appscrip

# Data science Assignment - 2

# In the given dataset there are 3 columns.
# Title
# Description
# Category 
# There are multiple categories. 
# Use this data to perform EDA and train a multi class classification model. 

# After training the model, use the below mentioned procedure to get new data.

# Now using the trained model to categorize the new data.

# Please use the following endpoint to get data.
# API endpoint - https://newsapi.org/docs/endpoints/everything
# API key - c0e2280d25ef481183f8b497ee36d692

# Use the following sources to call the API -
# The hindu, times of india and Hacker news.

# For more details to get data, you can refer the documentation from the above link.
# At Least gather 3k records.

# Dump the data in mongoDB. After data insertion , run the classification model on the data 
# and update the predicted category in the mongDB .

# Please share the jupyter notebook and the mongoDB dump. 

In [15]:
import os
os.chdir('D:\\Projects\\AppScrip')
import pandas as pd
import pandas as pd
import os
import pymongo
from pymongo import MongoClient
import pprint
import json
import pickle
from nltk.corpus import stopwords as Stopwords
stopword = list(Stopwords.words('english'))

In [ ]:
# Collect data and load to Mongo DB
# from newsapi import NewsApiClient
# newsapi = NewsApiClient(api_key='c0e2280d25ef481183f8b497ee36d692')
# articles = []
# for i in range(0,175):
#     top_headlines = newsapi.get_everything(sources='the-times-of-india,the-hindu,hacker-news',
#                                               language='en')
#     articles.extend(top_headlines['articles'])
# dummy_df = pd.DataFrame(articles)
# dummy_df = dummy_df[['title','description']]

In [55]:
channel = MongoClient(port = 27017)
select_database = channel.my_database
my_db = select_database.news_data

In [56]:
# data2json = json.loads(dummy_df.T.to_json()).values()
# select_database.news_data.insert(data2json)

In [110]:
# Retriving the data from MongoDB and transforming it to Pandas DataFrame
data_dict = {}
title = []
description = []
id1 = []
for x in my_db.find(): #find all the data in an data set and appending it to a dictionary
    id1.append(str(x['_id']))
    title.append(x['title'])    
    description.append(x['description'])

data_dict = {'_id': id1 ,'title': title,'description': description}
df = pd.DataFrame(data_dict, columns=['_id','title','description'])

In [111]:
symbols = ['!', '"', '#', '$', '%', '(', ')', '*', '+', ',',
           '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[',
           '\\', ']', '^', '_', '`', '{', '|', '}', '~','&',"'s"]


def text_cleaning(text):
    """
    text: enter the text/string
    returns: modified text/clean text
    """
    text = str(text)
    text =text.lower()
    for symbol in symbols: text = text.replace(symbol,'') # Replacing all symbols
    text = text.replace('  ', ' ')
    text = ' '.join(word for word in text.split() 
                    if word not in stopword and word.isalpha()==True and len(word)>2)
    return text

In [112]:
#TEXT PROCESSING FOR BAG OF WORDS AND TF-IDF 
#Lower case
df['process_title'] = df.title.str.lower()
df['process_description'] = df.description.str.lower()
#tag removal
df['process_title'] = df.title.str.replace('<[^<]+?>','')
df.description = df.description.str.replace('<[^<]+?>','')
#Removing the numbers
df['process_title'] = df.title.str.replace('\d+', '')
df['process_description'] = df.description.str.replace('\d+', '')
# TEXT CLEANING
df['process_title'] = df['title'].apply(text_cleaning)
df['process_description'] = df['description'].apply(text_cleaning)

In [114]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#load the count vectorizer and tfidf
filename = 'title_vector.sav'
count_vec = pickle.load(open(filename, 'rb'))
x_title = count_vec.transform(df.process_title)

filename = 'title_tfidf.sav'
tfidf_vec = pickle.load(open(filename, 'rb'))
x_title_tfidf = tfidf_vec.transform(x_title)

## Logistic regression model is used as the model has 86% of accuracy

In [115]:
category_convert = {0: 'entertainment',1: 'business', 2: 'general', 3: 'health', 4: 'science',
                    5: 'sports',6: 'technology', 7: 'automotive', 8: 'ent_regional',
                    9: 'ent_tv', 10: 'gaming', 11: 'national', 12: 'international'}

In [116]:
filename = 'title_lg.sav'
logistic_model = pickle.load(open(filename, 'rb'))
df['predicted'] = logistic_model.predict(x_title)
df.predicted = df.predicted.replace(category_convert)

In [117]:
df = df.drop(['process_title','process_description'],axis=1)

In [120]:
for index in range(df.shape[0]):
    my_db.update_one({"title":df.title.loc[index]},
                    {'$set':{"predict":df.predicted.loc[index]}})

In [122]:
for x in my_db.find():
    print(x)

{'_id': ObjectId('5da70d00fef38bb8d1e6d1af'), 'title': 'Sena expels 14 rebels; 36 party corporators resign', 'description': 'Even as the Shiv Sena expelled 14 rebels from the party recently, 36 corporators in Nashik resigned on Tuesday, giving another jolt to the party.The Sena and BJP saw a spurt in the number of rebels af', 'predict': 'general'}
{'_id': ObjectId('5da70d00fef38bb8d1e6d1b0'), 'title': 'City police to adopt 23 government schools', 'description': 'Steps will be taken to address issues relating to infrastructure, basic amenities: Police Commissioner', 'predict': 'general'}
{'_id': ObjectId('5da70d00fef38bb8d1e6d1b1'), 'title': 'Crazy Mohan’s birth anniversary to be observed', 'description': 'Carnatic music concert by Gayathri Girish will be based on his ‘Ramanayanam’.', 'predict': 'science'}
{'_id': ObjectId('5da70d00fef38bb8d1e6d1b2'), 'title': 'Traditional fishermen protest against Bill', 'description': '‘Stakeholders not consulted’', 'predict': 'general'}
{'_id': Objec

{'_id': ObjectId('5da70d00fef38bb8d1e6da91'), 'title': 'Singapore Business Federation team visits AU', 'description': 'V-C urges it to train students'}
{'_id': ObjectId('5da70d00fef38bb8d1e6da92'), 'title': 'Four-year-old girl dies as portion of building collapses in Virar', 'description': 'Fire brigade, police officials are undertaking search and rescue operations.'}
{'_id': ObjectId('5da70d00fef38bb8d1e6da93'), 'title': 'Arrest of CITU leaders sparks tension at Aurobindo Pharma', 'description': 'Large police force was deployed as a precautionary measure'}
{'_id': ObjectId('5da70d00fef38bb8d1e6da94'), 'title': 'SC judge rejects demand to recuse from land case', 'description': 'India News: NEW DELHI: The Supreme Court on Tuesday witnessed a sharp clash between Justice Arun Mishra and petitioners seeking his recusal in a land acquisition .'}
{'_id': ObjectId('5da70d00fef38bb8d1e6da95'), 'title': 'Capital unable to breathe easy as air quality remains poor', 'description': 'AQI up from 25